## Our initial analysis focused on evaluating the performance of the four selected classifiers using the complete DARWIN data set with all 450 features.

This approach served as our baseline, providing a fundamental understanding of each classifier's capability to differentiate between Alzheimer's patients and healthy controls.

### **This notebook contains the baseline for the Decision Tree**

In [2]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import KFold

# Fetch dataset 
darwin = fetch_ucirepo(id=732)

# Data (as pandas dataframes)
X = darwin.data.features
y = darwin.data.targets

X = X.drop(columns=['ID'])  # Remove ID column as it's not needed

# Best parameters for Decision Tree
params = {
    'criterion': 'gini', 
    'max_depth': 2, 
    'max_leaf_nodes': 2, 
    'min_samples_leaf': 2, 
    'min_samples_split': 2
}

# Lists to store performance metrics
all_accuracies = []
all_precisions = []
all_recalls = []
all_f1_scores = []
all_sensitivities = []
all_specificities = []

# Set up 10-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validation loop
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Ensure y_train and y_test are 1D
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    # Create Decision Tree classifier with the specified parameters
    clf_dt = DecisionTreeClassifier(**params, random_state=42)

    # Train the model
    clf_dt.fit(X_train, y_train)

    # Predict on test data
    y_pred = clf_dt.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary', pos_label='P')
    recall = recall_score(y_test, y_pred, average='binary', pos_label='P')
    f1 = f1_score(y_test, y_pred, average='binary', pos_label='P')

    all_accuracies.append(accuracy)
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1_scores.append(f1)

    # Calculate confusion matrix components
    cm = confusion_matrix(y_test, y_pred, labels=['H', 'P'])
    if cm.shape == (2, 2):  # Ensure it's a 2x2 matrix for binary classification
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    else:
        print(f"Unexpected confusion matrix shape: {cm.shape}")
        sensitivity = specificity = 0

    all_sensitivities.append(sensitivity)
    all_specificities.append(specificity)

# Calculate overall mean and standard deviation for metrics
final_metrics = {
    'mean_accuracy': np.mean(all_accuracies),
    'std_accuracy': np.std(all_accuracies),
    'mean_precision': np.mean(all_precisions),
    'std_precision': np.std(all_precisions),
    'mean_recall': np.mean(all_recalls),
    'std_recall': np.std(all_recalls),
    'mean_f1_score': np.mean(all_f1_scores),
    'std_f1_score': np.std(all_f1_scores),
    'mean_sensitivity': np.mean(all_sensitivities),
    'std_sensitivity': np.std(all_sensitivities),
    'mean_specificity': np.mean(all_specificities),
    'std_specificity': np.std(all_specificities),
}

# Print the final aggregated metrics
print("Final Aggregated Performance Metrics with 5-Fold Cross-Validation (Decision Tree):")
print(f"Mean Accuracy: {final_metrics['mean_accuracy']:.4f} ± {final_metrics['std_accuracy']:.4f}")
print(f"Mean Precision: {final_metrics['mean_precision']:.4f} ± {final_metrics['std_precision']:.4f}")
print(f"Mean Recall: {final_metrics['mean_recall']:.4f} ± {final_metrics['std_recall']:.4f}")
print(f"Mean F1 Score: {final_metrics['mean_f1_score']:.4f} ± {final_metrics['std_f1_score']:.4f}")
print(f"Mean Sensitivity: {final_metrics['mean_sensitivity']:.4f} ± {final_metrics['std_sensitivity']:.4f}")
print(f"Mean Specificity: {final_metrics['mean_specificity']:.4f} ± {final_metrics['std_specificity']:.4f}")


Final Aggregated Performance Metrics with 5-Fold Cross-Validation (Decision Tree):
Mean Accuracy: 0.8390 ± 0.0391
Mean Precision: 0.8147 ± 0.0824
Mean Recall: 0.9031 ± 0.0891
Mean F1 Score: 0.8497 ± 0.0404
Mean Sensitivity: 0.9031 ± 0.0891
Mean Specificity: 0.7839 ± 0.1020


In [3]:
all_accuracies

[0.9142857142857143,
 0.8285714285714286,
 0.8285714285714286,
 0.8,
 0.8235294117647058]

In [4]:
all_precisions

[np.float64(0.9473684210526315),
 np.float64(0.7619047619047619),
 np.float64(0.7391304347826086),
 np.float64(0.875),
 np.float64(0.75)]

In [5]:
all_recalls

[np.float64(0.9),
 np.float64(0.9411764705882353),
 np.float64(1.0),
 np.float64(0.7368421052631579),
 np.float64(0.9375)]

In [6]:
all_f1_scores

[np.float64(0.9230769230769231),
 np.float64(0.8421052631578947),
 np.float64(0.85),
 np.float64(0.8),
 np.float64(0.8333333333333334)]

In [7]:
all_sensitivities

[np.float64(0.9),
 np.float64(0.9411764705882353),
 np.float64(1.0),
 np.float64(0.7368421052631579),
 np.float64(0.9375)]

In [8]:
all_specificities

[np.float64(0.9333333333333333),
 np.float64(0.7222222222222222),
 np.float64(0.6666666666666666),
 np.float64(0.875),
 np.float64(0.7222222222222222)]